# The Brainy Bunch

Anthony Finch

-------

Hello and welcome to another exciting episode of *Yet Another Data Science Blog*.

Over the past few weeks, we've reviewed several topics in Unsupervised Learning.

* In [Is Bigger Really Better?](https://ajfinch123.github.io/ml/ai/unsupervised/compression/2020/08/31/is_bigger_better.html), we established a groundwork for this entire series on Unsupervised Learning by examining three core types of unsupervised tasks: dimensionality reduction, clustering, and anomaly detection.

* In [Honey, I Shrunk the Data](https://ajfinch123.github.io/ml/ai/unsupervised/compression/2020/09/08/shrunk_data.html), we explored two methods for dimensionality reduction: Principal Components Analysis and Neural Autoencoders.

* In [Acting Out, Part 1](https://ajfinch123.github.io/ml/ai/unsupervised/compression/word2vec/application/2020/09/13/acting_out.html), we examined a revolutionary application of dimensionality reduction called concept or word embedding.

## Disclaimer

I wanted to voice a quick disclaimer on this topic before we start.  This post is going to be pretty high-level, and a lot of the concepts here are explained just as well on Wikipedia or the SKLearn clustering documentation.

I'd like to include this particular post to make sure that we cover all of the unsupervised learning methods; however, clustering isn't a topic that gets me very excited.  There are some experiments in deep clustering that I've toyed around with - maybe we'll do a post on those at some point - but the core clustering algorithms have been around for many decades.  That isn't to say that clustering isn't useful - I've had to use clustering in real projects on several occasions (especially for customer segmentation).  However, I just don't find the core clustering algorithms like K-Means, Gaussian Mixture Models and DBSCAN all that complicated or interesting.

### Objective

Before we dive into the specifics, let's talk about what we want to accomplish.  Clustering is a popular unsupervised learning technique for creating groups of similar observations.  This can be useful for a number of different applications.  Frequently, clustering is used for preliminary data analysis, comparisons between output spaces, or for customer segmentation.  It can also be used to construct targets for unlabeled data or as a preliminary step in anomaly detection.  Generally speaking, clustering is going to be used on an unlabeled data set in order to help the modeler or analyst to understand some of the most common types of points in their data.

When using clustering algorithms, we are implicitly assuming that each cluster has a distinctive *Data Generating Process* (DGP).  This is essentially the same assumption that we make when we use supervised learning; the biggest difference is that, in clustering, we may not know the number of DGPs, or what they really mean.

Let's take a look at the now-famous SciKit-Learn Clustering comparison visualization.  This visualization displays several different datasets (along the columns) and the performance of different clustering algorithms on them (along the rows).  Each distinct color represents a different cluster.

![](./img/sklearn_cluster_comparison.png)

There are a few important things that we can take away from this visualization:

1. There are *a lot* of different clustering methods.
2. Clustering methods perform very differently on different types of datasets (as an example, just look at the performance of DBSCAN on the first few datasets and then look at its performance on the last dataset).
* Clustering methods can take wildly different amounts of time to run (just look at the bottom right-hand corner in each sub-figure; Affinity Propagation took almost 400x longer to run than K-Means on the first dataset).

# So Which One is Best?

This question is frustratingly complicated.  It feels so intuitive to just run the best clustering algorithm and move on.  Alternatively, if there are many different competing algorithms, why not just run all of them and see which one is best?

The problem here is that we may not have an esternal optimization metric that we can employ.  Let's take two contrasting examples for the same application.  Suppose that we're trying to identify customer segments.  In the first example, we're identifying customers who may want to purchase different products.  In that case, we could cluster our customers based on their demographic (or other) variables, and then see how well customer purchasing behavior aligned with those clusters.  We don't want a predictive model of customer purchases, but we can employ that data as a rough evaluation metric.  In this case, we would have an external metric that we could examine after running several different algorithms.

As an alternative, consider an application where we're trying to identify customer segments that may be influenced by different advertisements.  If we don't have prior data on how customers have reacted our previous ads, we can't really evaluate on some external metric.  This kind of application, where we don't really have an external metric, is surprisingly common.  In this case, we really need to understand how different algorithms may behave.

Of course, it's also important to understand these differences because it can be inefficient to run every single possible algorithm and see which ones work best.  This can also lead to overfitting, if we don't retain a validation dataset.

# Clustering Types

There are *many* different clustering algorithms (even more, in fact, than those displayed here).  The key to using this method successfuly is to know your options and to understand your dataset.  While there are many different specific implementations, there are a few rules you can use to think about what kinds of clusters you want.

### Parameter-Learning Methods

The first and (possibly) most intuitive family of clustering algorithms is centroid- or parameter-learning algorithms.  The most popular example is K-Means clustering, followed by Gaussian Mixture Models.

These methods attempt to learn a set of parameters from the data; after that task is complete, the data can be discarded.  In K-Means clustering, we're trying to learn the coordinates for a set of 'centroids', which are basically canonical representatives of the different classes.  We can think of this as learning to look for the stereotypical example of a group of points.  Gaussian Mixture Models take this a step further and look for both a stereotypical example of a group and also the shape of the variance around that group.

Taking the example of customer segmentation, we might look at a group of 10,000 customers and decide that we want 2 clusters.  When our algorithm has finished running, we note that there are two stereotypical customer representations.  Our first stereotypical customer is a young professional that spends most of their time on the computer, and our second stereotypical customer is an older retiree that enjoys reading.  If our goal was to advertise to these different groups, we might choose to target our first cluster with email or Facebook ads, while we would use mailing campaigns to target our second cluster.

There are several advantages to this methodd of identifying clusters.  We can typically select our desired number of clusters, and these algorithms are extremely efficient to compute and retain.  Unfortunately, these methods can also lead to a false sense of security.  Depending on the shape of the underlying data (as above), it's possible that we'll lose a significant amount of granularity and accuracy by reducing an entire population down to a single stereotypical representation.  Moreover, we may end up with many customers getting misallocated to the wrong cluster, since identified clusters may not align with real customer segments (just look at the performance of these methods on the first SKLearn dataset in the graph above).

**Advantages**

* Easy to use and understand.
* Small storage requirements.
* Works well on larger datasets (K-Means; GMM does not scale).
* Choose number of clusters.

**Disadvantages**

* Typically cannot identify complex cluster shapes
* Need to know how many clusters you want before clustering.

### Density and Connection Techniques

In contrast to a technique where we discover stereotypical examples, there is a large family of algorithms that allocates points to clusters based on their positions relative to other points.  In hierarchical clustering, we start with every point in its own cluster, and then clusters are iteratively combined based on a distance rule.

Alternatively, DBSCAN and similar algorithms examine the density of points and try to construct cut-points along low-density regions that divide high-density ones.  This can allow us to construct clusters that reflect the data much more intuitively (see, for instance, the performance of DBSCAN on the first dataset).  I like to visualize this as a diorama.  Where there are a lot of points, we see a mountain on the diorama.  Where there are very few points, we see a valley.  We start filling our diorama with water, which naturally settles in the valleys (the low-density regions).  These rivers, lakes, or other bodies of water will constitute the divides between our clusters in a density-based algorithm.

Ultimately, all of these methods use the connections between data points to construct a *region* where a cluster applies, as opposed to constructing some kind of summary representation of that cluster.  Typically, this means that we have to retain the data in order to obtain a cluster label for new data points (unless we do something clever).

The biggest problem with these algorithms is that they don't tend to work particularly well on large datasets (with more points, we're less likely to have very low-density regions, meaning that we can end up with all of our points being connected) and they don't work well when data has many dimensions (since data is likely to be separated along so many axes).

**Advantages**

* Works on complicated shapes.

**Disadvantes**

* Doesn't work as well with big/wide data.
* Bigger storage requirements/doesn't really allow for prediction.
* Can end up with degenerate clusters (e.g. hundreds of small clusters or one single giant cluster).

# Tricks of the Trade

Here's the thing: I've never seen anyone use anything other than K-Means, Gaussian Mixture, or DBSCAN in practice.  Actually, most applications I've seen really just boil down to K-Means.

I know somone is going to get on my case for that, and I'm sure that isn't true in general.  However, I do think that K-Means is the go-to technique for clustering.  The biggest problem with K-Means is that it doesn't work well for complex data shapes.  Is there anything that we can do to leave us with the advantages of K-Means, while still giving us some of the better performance of other algorithms on data coming from complicated shapes?

Enter: dimensionality reduction and feature compression.

As demonstrated in [Honey, I Shrunk the Data](https://ajfinch123.github.io/ml/ai/unsupervised/compression/2020/09/08/shrunk_data.html), dimensionality reduction is capable of capturing highly complex relationships between variables and embedding those into a smaller, simpler feature space.  This is exactly the motivation behind [Spectral Clustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html#sklearn.cluster.SpectralClustering), which uses dimensionality before a simple K-means (or alternative) clustering algorithm to create highly effective groups.  The 